In [ ]:
%config InlineBackend.figure_format='png'
%matplotlib inline

In [ ]:
from netCDF4 import Dataset
import numpy as np
from scipy.interpolate import interp1d

In [ ]:
template = Dataset('../create_wrfinput_r/wrfinput_d01_new.nc', 'r+')
met_data = Dataset('../metgrid4wrfinput/met_em.d01.2018-08-01_00:00:00.nc', 'r')

In [ ]:
met_data_vars = list(met_data.variables.keys())
for a, b, c, d in zip(
    met_data_vars[::4], met_data_vars[1::4], met_data_vars[2::4], met_data_vars[3::4]
):
    print(("{:<16s}" * 4).format(a, b, c, d))


In [ ]:
template_vars = list(template.variables.keys())
for a, b, c, d in zip(
    template_vars[::4], template_vars[1::4], template_vars[2::4], template_vars[3::4]
):
    print(("{:<16s}" * 4).format(a, b, c, d))

In [ ]:
tslb = template.variables['TSLB']
smois = template.variables['SMOIS']
tmn = template.variables['TMN']
tsk = template.variables['TSK']
tsk_met = met_data.variables['SKINTEMP']
st_met = met_data.variables['ST']
sm_met = met_data.variables['SM']

In [ ]:
print(tslb[:].dtype)

In [ ]:
print(np.ma.count_masked(sm_met[:]))

In [ ]:
dzs = template.variables['DZS']
zs = template.variables['ZS']
zs_1d = zs[0,:]

In [ ]:
print(zs_1d)

In [ ]:
print(dzs[0,:])

In [ ]:
zsi_met = met_data.variables['SOIL_LAYERS']
zsi_met_1d = zsi_met[0,:,0,0]
zsi_met_1d_flipped = zsi_met_1d[::-1]

In [ ]:
print(zsi_met_1d_flipped)

In [ ]:
nt, nz, ny, nx = tslb.shape
print(nt, nz, ny, nx)

In [ ]:
st_met_expanded = np.zeros((nt,nz+2, ny, nx), dtype=tslb[:].dtype)

In [ ]:
st_met_expanded[0,0,:,:] = tsk_met[0,:,:]
st_met_expanded[0,-1,:,:] = tmn[0,:,:]
st_met_expanded[0,1:-1,:,:] = st_met[0,::-1,:,:]

In [ ]:
sm_met_expanded = np.zeros((nt,nz+2, ny, nx), dtype=tslb[:].dtype)

In [ ]:
sm_met_expanded[0,0,:,:] = sm_met[0,-1,:,:]
sm_met_expanded[0,-1,:,:] = sm_met[0,0,:,:]
sm_met_expanded[0,1:-1,:,:] = sm_met[0,::-1,:,:]

In [ ]:
zsi1 = np.insert(zsi_met_1d_flipped, 0, 0.0)
z_met_expanded = np.zeros(sm_met_expanded.shape[1], dtype = zs.dtype)
z_met_expanded[-1] = 3.0
z_met_expanded[1:-1] = (zsi1[:-1] + zsi1[1:])/2.0/100.0

In [ ]:
print(z_met_expanded)

In [ ]:
f = interp1d(z_met_expanded, st_met_expanded, kind='linear', axis=1)
tslb = f(zs)
del f

In [ ]:
f = interp1d(z_met_expanded, sm_met_expanded, kind='linear', axis=1)
smois = f(zs)
del f